# Extracting sign distributions

This notbook contains code for:
- importing necessary libraries
- loading stored WTs
- creating random sparse networks
- Extracting sign distributions from WTs
- Extracting sign distributions from non-WTs
- saving extracted distributions in files

### Requirements:

In [1]:
# importing necessary libraries and the cnn architecture I defined

from cnn_architecture import CNN2Model
from utils import *

import tensorflow_datasets as tfds
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random

In [2]:
def load_and_prep_cifar(batch_size, shuffle_size):
    # load data set
    (train_ds, test_ds), ds_info = tfds.load(name="cifar10", split=["train","test"], as_supervised=True, with_info=True)
    # tfds.show_examples(train_ds, ds_info)
    
    def prepare_cifar10_data(ds):
        #convert data from uint8 to float32
        ds = ds.map(lambda img, target: (tf.cast(img, tf.float32), target))
        #sloppy input normalization, just bringing image values from range [0, 255] to [-1, 1]
        ds = ds.map(lambda img, target: ((img/128.)-1., target))
        #create one-hot targets
        ds = ds.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
        #cache this progress in memory, as there is no need to redo it; it is deterministic after all
        ds = ds.cache()
        #shuffle, batch, prefetch
        ds = ds.shuffle(shuffle_size).batch(batch_size).prefetch(2)
        #return preprocessed dataset
        return ds
    
    # prepare data
    train_dataset = train_ds.apply(prepare_cifar10_data)
    test_dataset = test_ds.apply(prepare_cifar10_data)
    
    return train_dataset, test_dataset

In [3]:
# extract sign distribution from one layer

def sign_distribution_layer(this_layer, next_layer):
    '''
    columns: 
    ["prune_rate_in", "prune_rate_out", "pos_in", "pos_out", "neg_in", "neg_out", "sign_rate_in", "sign_rate_out"]
    '''
    sign_distr = pd.DataFrame()
    unconnected_neurons = {"no_incomming": 0 , "no_outgoing" : 0, "no_total_conncetions" : 0}
    for this_neuron, next_neurons, i in zip(this_layer.T, next_layer, range(np.shape(this_layer)[0])):
        
        this_unpruned = this_neuron[this_neuron != 0.0]
        next_unpruned = next_neurons[next_neurons != 0.0]
        
        # only include neurons that receive and propagate input
        if len(this_unpruned)*len(next_unpruned) != 0:
            
            dic = {} 
            dic["pos_in"] = len(this_unpruned[this_unpruned>0])
            dic["pos_out"] = len(next_unpruned[next_unpruned>0])
            dic["neg_in"] = len(this_unpruned[this_unpruned<0])
            dic["neg_out"] = len(next_unpruned[next_unpruned<0])
            dic["prune_rate_in"] = 1 - (len(this_unpruned)/len(this_neuron))
            dic["prune_rate_out"] = 1 - (len(next_unpruned)/len(next_neurons))
            dic["sign_rate_in"] = dic["pos_in"]/len(this_unpruned)   
            dic["sign_rate_out"] = dic["pos_out"]/len(next_unpruned)                               

            df = pd.DataFrame(data = dic, index = [i])
            sign_distr = pd.concat([sign_distr, df], axis=0)
        
        # store information about inconnected neurons
        else :
            
            if len(this_unpruned) == 0:
                if len(next_unpruned) ==0:
                    unconnected_neurons["no_total_conncetions"] = unconnected_neurons["no_total_conncetions"] + 1
                else:
                    unconnected_neurons["no_incomming"] = unconnected_neurons["no_incomming"] + 1
            else:
                unconnected_neurons["no_outgoing"] = unconnected_neurons["no_outgoing"] + 1
        
    return sign_distr, unconnected_neurons

In [4]:
# extract sign distribution form two layers

def sign_distribution_layers(layer1, layer2, layer3):
    
    sign_distr1, unconnected_neurons1 = sign_distribution_layer(layer1, layer2)
    sign_distr2, unconnected_neurons2 = sign_distribution_layer(layer2, layer3)
    sign_distr1["layer"] = "dense1"
    sign_distr2["layer"] = "dense2"
    sign_distr_both = pd.concat([sign_distr1, sign_distr2], axis=0)
    
    unconnected_neurons1["layer"] = "dense1"
    unconnected_neurons2["layer"] = "dense2"

    return sign_distr_both, unconnected_neurons1, unconnected_neurons2

## Extracting sign distributions and storing them as files:

In [5]:
wts_per_dataset = 15

In [6]:
# extract sign distributions

def get_sign_distr(dataset, pruning_name, n):

    # collect statistics of unconnected neurons
    unconnected_statistics = pd.DataFrame()
    
    # make a model to load the weights into
    train_dataset, test_dataset = load_and_prep_cifar(batch_size=60, shuffle_size=512)
    model = CNN2Model()
    model(list(train_dataset)[0][0])
    for i in tqdm(range(n), leave=False, desc=f"sign_distributions of {dataset}"):
        
        # get WT weights
        model.load_weights(f"1b WTs/WT_{dataset}_{pruning_name}_{i}")    
        weights_wt = model.get_weights()
        # get sign distribution
        sign_distr_wt, unconnected_neurons1, unconnected_neurons2 = sign_distribution_layers(weights_wt[4], weights_wt[6], weights_wt[8])
        # store sign distribution
        sign_distr_wt.to_csv(f'2b Sign distributions/{dataset}_{pruning_name}_{i}_sign_distr.csv', index=False)
        
        
        #collect unconnected neurons statistics
        unconnected_neurons1["model"] = f"WT_{dataset}_{pruning_name}_{i}"
        unconnected_neurons2["model"] = f"WT_{dataset}_{pruning_name}_{i}"
        df1 = pd.DataFrame(data = unconnected_neurons1, index = [i])
        df2 = pd.DataFrame(data = unconnected_neurons2, index = [i])
        unconnected_statistics = pd.concat([unconnected_statistics, df1, df2], axis=0)
        
    unconnected_statistics.to_csv(f'2b Sign distributions/{dataset}_{pruning_name}_unconnected_statistic.csv', index=False)

In [7]:
# store sign distributions of Wts

for dataset in ["SVHN","CINIC","CIFAR"]:
    get_sign_distr(dataset,"IMP",wts_per_dataset)

### Generating randomly pruned sign distributions

In [8]:
# implement random pruning of a weight matrix according to a pruning rate

def random_pruning(model,pruning_rates):
    
    weights = model.get_weights()
    new_weights = []
    
    for layer, p_rate in zip(weights[::2], pruning_rates):
        shape = np.shape(layer)
        layer = layer.flatten()
        number_to_prune = int(len(layer) * p_rate)
        pruning_indexi = random.sample(range(len(layer)),number_to_prune)
        layer[pruning_indexi] = 0.0
        layer = np.reshape(layer,shape)
        new_weights.append(layer)
        
    weights[::2] = new_weights
    model.set_weights(weights)
    weights = model.get_weights()
    
    return model

In [9]:
# get pruning rates from imp model
train_dataset, test_dataset = load_and_prep_cifar(batch_size=60, shuffle_size=512)
model_cifar_imp = CNN2Model()
model_cifar_imp(list(train_dataset)[0][0])
model_cifar_imp.load_weights("1b WTs/WT_CIFAR_IMP_0")
pruning_rates = get_pruning_rates(model_cifar_imp.get_weights()[::2])

unconnected_statistics = pd.DataFrame()

# generate random sparse networks
for i in range(wts_per_dataset):
    
    # randomly prune a model
    random_model = CNN2Model()
    random_model(list(train_dataset)[0][0])
    random_model = random_pruning(random_model,pruning_rates)
    weights_rsn = random_model.get_weights()
    
    # store sign distributions of random sparse networks
    sign_distr_random, unconnected_neurons1, unconnected_neurons2  = sign_distribution_layers(weights_rsn[4], weights_rsn[6], weights_rsn[8])
    sign_distr_random.to_csv(f"2b Sign distributions/RSN_{i}_sign_distr.csv", index=False)
    
    #collect unconnected neurons statistics
    unconnected_neurons1["model"] = f"RSN_{i}"
    unconnected_neurons2["model"] = f"RSN_{i}"
    df1 = pd.DataFrame(data = unconnected_neurons1, index = [i])
    df2 = pd.DataFrame(data = unconnected_neurons2, index = [i])
    unconnected_statistics = pd.concat([unconnected_statistics, df1, df2], axis=0)

# store unconnected statistics
unconnected_statistics.to_csv(f'2b Sign distributions/RSN_unconnected_statistic.csv', index=False)

## Pruning rates

In [12]:
train_dataset, test_dataset = load_and_prep_cifar(batch_size=60, shuffle_size=512)
model_cifar_imp = CNN2Model()
model_cifar_imp(list(train_dataset)[0][0])
model_cifar_imp.load_weights("1b WTs/WT_CIFAR_IMP_0")

In [14]:
# return the pruning rates of the conv and dense layers
print("conv rate (without biases): ", get_pruning_rate(model_cifar_imp.get_weights()[0:4:2]))
print("dense rate (without biases): ", get_pruning_rate(model_cifar_imp.get_weights()[4::2]))
print("conv rate (with biases): ", get_pruning_rate(model_cifar_imp.get_weights()[0:4:]))
print("dense rate (with biases): ", get_pruning_rate(model_cifar_imp.get_weights()[4::]))
print("pruning rates without biases each layer: ", get_pruning_rates(model_cifar_imp.get_weights()[::2]))
print("pruning rate total with biases: ", get_pruning_rate(model_cifar_imp.get_weights()))
print("pruning rate total without biases: ", get_pruning_rate(model_cifar_imp.get_weights()[::2]))

conv rate (without biases):  0.6794931592039801
dense rate (without biases):  0.9176456925675676
conv rate (with biases):  0.6772469008264463
dense rate (with biases):  0.9175333257329128
pruning rates without biases each layer:  [0.6799768518518519, 0.6794704861111112, 0.9284100532531738, 0.259185791015625, 0.13789062500000004]
pruning rate total with biases:  0.9153704562118372
pruning rate total without biases:  0.9155087942502567
